In [1]:
import json


In [2]:
with open('mascqa-eval.json','r') as f:
    questions_file = json.load(f)
f.close()

In [3]:
import pandas as pd
import os

df = pd.DataFrame(questions_file)

In [4]:
import ollama

def ask_questions(questions):
    """
    Ask a list of questions to an LLM model like gpt-3.5-turbo and get the answers in a list format.

    Parameters:
    questions (list): A list of questions to ask the model.

    Returns:
    list: A list of answers provided by the model.
    """
    
    prompt = "Solve the following question. Write the correct answer inside a list at the end."
    
    answers = []
    for question in questions:
        response = ollama.chat(model='llama3:70b', #ID 786f3184aec0 
                           messages=[
                {"role": "assistant", "content":prompt},
                {"role": "user", "content": question}
            ])
        
        # Extract the response from the model
        model_response = response['message']['content']
        answers.append(model_response)

    return answers


# Example usage:
questions = [
    "What is the capital of France?",
    "What is 2 + 2?",
    "Who wrote 'To Kill a Mockingbird'?"
]

answers = ask_questions(questions)
answers

['The capital of France is:\n\n* Paris',
 'The answer to 2 + 2 is:\n\n* 4',
 'The correct answer is:\n\n* Harper Lee']

In [5]:
for i in questions_file.keys():
    newpath = f'./answer_llama_3_70b/{i}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    questions = df[f"{i}"]["questions"]
    answers = ask_questions(questions)
    for j in range(len(answers)):
        a = df[f"{i}"]["qids"][j]
        with open(f"./{newpath}/{a}.txt",'w') as file:
            file.write(answers[j])
    